In [ ]:
! pip3 isntall openpyxl
! pip3 install langchain_community
! pip3 install langchain

In [ ]:
import pandas as pd
from IPython.display import Markdown, display
df = pd.read_excel('premier_league_tweets.xlsx')
df.head()

In [ ]:
df['player_name'].unique()

In [ ]:
df['player_name'].value_counts()

In [ ]:
contexto = " "
for jugador, tweet in zip(df['player_name'], df['tweet']):
    contexto += f"""
    +++++++++++++++++++++++++++++++++++++++
    Tweet sobre {jugador}:
    {tweet}
    +++++++++++++++++++++++++++++++++++++++
    """

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain_community.chat_models import ChatOCIGenAI

# OCI Configuration
COMPARTMENT_ID = "AQUI VA TU COMPARTMENT"
AUTH_TYPE = "API_KEY"
CONFIG_PROFILE = "DEFAULT"
ENDPOINT = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"
SYSTEM_PROMPT = """
Eres un narrador deportivo experto especializado en crear narrativas concisas y dinámicas a partir de información breve. Tu tarea es analizar tweets sobre deportistas y transformarlos en comentarios deportivos profesionales y entretenidos.

INSTRUCCIONES:
1. Cuando recibas el nombre de un jugador, localiza en el contexto los tweets relacionados con ese atleta.
2. Analiza eficientemente el contenido limitado de estos tweets, extrayendo:
   - Menciones de rendimiento o estadísticas
   - Reacciones a partidos o competencias
   - Opiniones sobre habilidades o momentos destacados
   - Cualquier información relevante sobre su carrera o situación actual

3. Genera una narrativa deportiva concisa pero impactante que:
   - Sintetice la esencia de la información disponible en los tweets
   - Utilice lenguaje dinámico y expresivo del periodismo deportivo
   - Mantenga un tono profesional pero accesible
   - Contextualice la información dentro del panorama deportivo actual
   - Se adapte al volumen limitado de información disponible

4. Mantén tus narrativas breves y directas, ideales para formatos como:
   - Comentarios rápidos entre jugadas
   - Resúmenes post-partido
   - Introducciones de segmentos deportivos
   - Publicaciones en redes sociales

5. Agregar una seccion de "Lo que la gente piensa" donde se muestren citas de los tweets encontrados. Solo citalos, no hagas
comentarios al respecto. Divide esta seccion en positivos y negativos

Si no encuentras nada de dicho jugador, di que no puedes ayudar debido a la ausencia de informacion

Si los tweets contienen información contradictoria o poco clara, enfócate en los elementos más consistentes o mencionados con mayor frecuencia.

Tu objetivo es transformar fragmentos breves de información en narrativas deportivas profesionales que capturen la atención y transmitan lo esencial del mensaje original.

Responde siempre usando markdown
"""

jugador = "AQUI PON EL NOMBRE DE ALGUN JUGADOR"

HUMAN_MESSAGE = f"""
Utilizando la siguiente informacion como contexto: {contexto},
genera la narrativa de este jugador: {jugador}
"""

model_kwargs = {
        "temperature": 1.0,
        "max_tokens": 1200,
        "top_p": 0.7
    }

llm = ChatOCIGenAI(
            model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyajqi26fkxly6qje5ysvezzrypapl7ujdnqfjq6hzo2loq",
            service_endpoint=ENDPOINT,
            compartment_id=COMPARTMENT_ID,
            provider="meta",
            auth_type=AUTH_TYPE,
            auth_profile=CONFIG_PROFILE,
            model_kwargs=model_kwargs
        )

messages = [SystemMessage(content=SYSTEM_PROMPT), HumanMessage(content=HUMAN_MESSAGE)]
   
  
response = llm.invoke(messages)

display(Markdown(response.content))